In [1]:
!pip install torch_scatter torcheeg torch_geometric -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.1/295.1 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 63.6 MB/s eta 0:00:00

In [2]:
import torch
import torch.nn as nn
import torch.nn.utils as utils
from torch.utils.data import DataLoader, Subset,WeightedRandomSampler
from torcheeg.models import CCNN
from torcheeg import transforms
from torcheeg.transforms import ToGrid
from torcheeg.datasets import SEEDIVDataset,SEEDIVFeatureDataset
from torcheeg.datasets.constants import SEED_IV_CHANNEL_LOCATION_DICT
from torcheeg.transforms import ToG
from torcheeg.datasets.constants import SEED_IV_ADJACENCY_MATRIX
from torcheeg.models import DGCNN
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import ReduceLROnPlateau 
# --- THE MAIN SUBJECT LOOP ---

import torch_geometric.loader as geom_loader # Special loader for graphs
import copy
import scipy.signal as signal
import random
import numpy as np

In [3]:
import shutil
import os

if os.path.exists('./tmp_out/seed_iv_features'):
    shutil.rmtree('./tmp_out/seed_iv_features')
    print("Old cache deleted. Data will be re-processed.")

In [4]:
# 1. Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from torcheeg.datasets.constants import SEED_IV_CHANNEL_LOCATION_DICT
from torcheeg.transforms import ToGrid

dataset = SEEDIVFeatureDataset(
    io_path='./tmp_out/seed_iv_features',
    root_path='/kaggle/input/seed-iv/eeg_feature_smooth',
    feature=['de_LDS'],
    num_worker=0,
    offline_transform=transforms.Compose([
        ToGrid(SEED_IV_CHANNEL_LOCATION_DICT),
        transforms.MinMaxNormalize(),
        transforms.Lambda(lambda x: torch.tensor(x).float())
        
    ]),
    label_transform=transforms.Compose([
        transforms.Select('emotion')
    ])
)


[2025-12-11 14:23:09] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./tmp_out/seed_iv_features.
[2025-12-11 14:23:09] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/45 [00:00<?, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 1it [00:00,  8.91it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 17it [00:00, 92.54it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 35it [00:00, 129.75it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 54it [00:00, 152.20it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_20151111.mat]: 73it [00:00, 165.07it/s]
[RECORD /kaggle/input/seed-iv/eeg_feature_smooth/1/4_2015111

In [6]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

In [7]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42) 

In [8]:
subjects = sorted(dataset.info['subject_id'].unique()) # Get list of all 15 subjects

In [9]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in loader:
        x = batch[0].to(device)
        y = batch[1].to(device)


        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = out.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    return total_loss / len(loader), correct / total

In [10]:
def evaluate(model, loader, criterion):
    model.eval()
    
    total_loss = 0.0  
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in loader:
            x = batch[0].to(device)
            y = batch[1].to(device)

            out = model(x)
            
            loss = criterion(out, y)
            total_loss += loss.item()

            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return total_loss / len(loader), correct / total

In [11]:
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

In [12]:
all_subject_accuracies = []

for subject_id in subjects:
    print(f"\nProcessing SUBJECT: {subject_id}")
    
    # --- Data Split by Trials ---
    sub_df = dataset.info[dataset.info['subject_id'] == subject_id]
    all_trials = list(range(1, 25))
    random.seed(42)
    test_trials = random.sample(all_trials, 5)
    train_trials = [t for t in all_trials if t not in test_trials]

    train_indices = sub_df[sub_df['trial_id'].isin(train_trials)].index.tolist()
    test_indices = sub_df[sub_df['trial_id'].isin(test_trials)].index.tolist()

    train_set = Subset(dataset, train_indices)
    test_set = Subset(dataset, test_indices)

    # --- Sampler for Class Imbalance ---
    y_train_indices = train_set.indices
    raw_labels = dataset.info.iloc[y_train_indices]['emotion'].values
    class_counts = np.bincount(raw_labels)
    class_weights = 1. / class_counts
    sample_weights = [class_weights[y] for y in raw_labels]

    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

    train_loader = DataLoader(train_set, batch_size=64, sampler=sampler, num_workers=0)
    test_loader = DataLoader(test_set, batch_size=64, shuffle=False, num_workers=0)

    # --- Model ---
    model = CCNN(
        num_classes=4,
        in_channels=5,        
        grid_size=(9, 9),     
        dropout=0.3
    ).to(device)
    model.apply(weights_init)

    # --- Criterion & Optimizer ---
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    epochs = 75
    best_acc = 0.0

    # --- Training Loop ---
    for epoch in range(epochs):
        model.train()
        train_loss_sum = 0
        correct = 0
        total = 0

        for batch in train_loader:
            x, y = batch
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

            train_loss_sum += loss.item()
            preds = out.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

        avg_train_loss = train_loss_sum / len(train_loader)
        train_acc = correct / total

        val_loss, val_acc = evaluate(model, test_loader, criterion)
        scheduler.step(val_loss)

        is_best = False
        if val_acc > best_acc:
            best_acc = val_acc
            is_best = True
            
            save_path = os.path.join(save_dir, f"best_model_subject_{subject_id}.pth")
            torch.save(model.state_dict(), save_path)

        

        if is_best or (epoch + 1) % 5 == 0:
            status = "(*) BEST" if is_best else ""
            print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f} {status}")

    all_subject_accuracies.append(best_acc)
    print(f"Subject {subject_id} FINAL BEST ACC: {best_acc*100:.2f}%")

# --- Final Summary ---
print("\n" + "="*50)
print("FINAL RESULTS SUMMARY")
print("="*50)
final_avg_acc = np.mean(all_subject_accuracies)
final_std_acc = np.std(all_subject_accuracies)
print(f"Subjects: {len(subjects)} | Average Accuracy: {final_avg_acc*100:.2f}% | Std: {final_std_acc*100:.2f}")
print(f"Accuracies List: {[round(x*100,2) for x in all_subject_accuracies]}")


Processing SUBJECT: 1


/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 1/75 | Train Loss: 1.5693 | Val Loss: 1.4481 | Train Acc: 0.263 | Val Acc: 0.125 (*) BEST
Epoch 2/75 | Train Loss: 1.3871 | Val Loss: 1.3785 | Train Acc: 0.272 | Val Acc: 0.412 (*) BEST
Epoch 4/75 | Train Loss: 1.3814 | Val Loss: 1.3920 | Train Acc: 0.287 | Val Acc: 0.469 (*) BEST
Epoch 5/75 | Train Loss: 1.3784 | Val Loss: 1.3511 | Train Acc: 0.292 | Val Acc: 0.382 
Epoch 10/75 | Train Loss: 1.3798 | Val Loss: 1.4115 | Train Acc: 0.285 | Val Acc: 0.125 
Epoch 15/75 | Train Loss: 1.3690 | Val Loss: 1.4045 | Train Acc: 0.321 | Val Acc: 0.241 
Epoch 20/75 | Train Loss: 1.3487 | Val Loss: 1.3827 | Train Acc: 0.395 | Val Acc: 0.206 
Epoch 25/75 | Train Loss: 1.3355 | Val Loss: 1.3903 | Train Acc: 0.430 | Val Acc: 0.312 
Epoch 30/75 | Train Loss: 1.3259 | Val Loss: 1.3745 | Train Acc: 0.438 | Val Acc: 0.333 
Epoch 35/75 | Train Loss: 1.3217 | Val Loss: 1.3746 | Train Acc: 0.478 | Val Acc: 0.307 
Epoch 40/75 | Train Loss: 1.3151 | Val Loss: 1.3925 | Train Acc: 0.524 | Val Acc: 0.312 
E